# How-to Log Sum Exp

Using flash-attention intensively you will at some point hear about `lse` values being returend. `lse` stands for log sum exp can be used to compute softmax (and thereby also attention) in a blockwise and stable fashion. This notebook aims to explain how this works.

Let's start by defining a naive softmax function ..

In [103]:
from typing import Tuple, Sequence
import torch
import math

def naive_softmax(x: torch.Tensor) -> torch.Tensor:
    return x.exp() / x.exp().sum()

... and verify that its output matches the output of the official `torch.softmax()` function.

In [104]:
x = torch.randn(10)  # generate normally distributed random numbers
a = torch.softmax(x, dim=-1) # reference output
b = naive_softmax(x) # our naive version

print("a", a)
print("b", b)
print("allclose", torch.allclose(a, b, atol=1e-6))

a tensor([0.1621, 0.0160, 0.1076, 0.0987, 0.0044, 0.0168, 0.0487, 0.3812, 0.0206,
        0.1438])
b tensor([0.1621, 0.0160, 0.1076, 0.0987, 0.0044, 0.0168, 0.0487, 0.3812, 0.0206,
        0.1438])
allclose True


Our naive softmax function has a problem with numerical stability when it get input vectors with larger elements:

In [105]:
naive_softmax(x * 100)

tensor([nan, 0., 0., 0., 0., 0., 0., nan, 0., nan])

But before we will look into how to fix this let's first look at how a blockwise computation of softmax can be implemented with `naive_softmax()`. We generate a vector and split it into two chunks of equal size and compute softmax of the chunks individually.

In [106]:
x = torch.randn(10)

x1,x2 = torch.chunk(x, 2)
s1 = naive_softmax(x1)
s2 = naive_softmax(x2)

print("We have:")
print(f"s1 = {s1}")
print(f"s2 = {s2}")

target = naive_softmax(x)
print("We want:")
print(f"target = {s}")

We have:
s1 = tensor([0.0534, 0.2454, 0.4393, 0.1991, 0.0627])
s2 = tensor([0.4687, 0.0210, 0.1900, 0.0526, 0.2677])
We want:
target = tensor([0.0586, 0.0118, 0.0048, 0.0119, 0.0144, 0.0147, 0.0370, 0.0435, 0.0064,
        0.0116, 0.0723, 0.0200, 0.0224, 0.1518, 0.3166, 0.0333, 0.0943, 0.0225,
        0.0115, 0.0405])


If we look at `naive_softmax()` we note that its output has been divided by `x.exp().sum()`. We can call this the "sum exp" value (note the similarity to "log sum exp") and we can use it to "undo" the softmax normalization and thereby compute combine multiple softmax chunks if we have this vaue for each chunk.

In [107]:
se_x1 = x1.exp().sum()
se_x2 = x2.exp().sum()
s1_corrected = s1 * se_x1 / (se_x1 + se_x2)
s2_corrected = s2 * se_x2 / (se_x1 + se_x2)

print("After correction with help of lse values:")
s_combined = torch.cat([s1_corrected, s2_corrected])
print("s_combined", s_combined)

print("allclose(s_combined, target):", torch.allclose(s_combined, target))


After correction with help of lse values:
s_combined tensor([0.0187, 0.0860, 0.1540, 0.0698, 0.0220, 0.3044, 0.0137, 0.1234, 0.0341,
        0.1738])
allclose(s_combined, target): True


... but is this helpful at all? Yes, and it becomes more obivous when we realize that we can return this value from our softmax function and we can do the correction in a blockwise fashion in a loop by accumulating the lse value:

In [108]:
def naive_softmax2(x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
    exp_sum = x.exp().sum()
    return x.exp() / exp_sum, exp_sum


def naive_blockwise_softmax(blocks: Sequence[torch.Tensor]) -> torch.Tensor:
    out = []
    exp_sum = 0
    for block in blocks:
        block_softmax, block_sum_exp = naive_softmax2(block)
        for o in out:
            o *= exp_sum / (exp_sum + block_sum_exp)
        
        out.append(block_softmax * block_sum_exp / (block_sum_exp + exp_sum))
        exp_sum += block_sum_exp
        
    return torch.cat(out)
    

x_long = torch.randn(20)
chunks = torch.chunk(x_long, 4)
print(naive_blockwise_softmax(chunks))
print(torch.softmax(x_long, dim=-1))


tensor([0.0045, 0.0069, 0.0380, 0.0068, 0.0475, 0.1233, 0.0354, 0.0832, 0.0463,
        0.0229, 0.0486, 0.0456, 0.0153, 0.0123, 0.0321, 0.0352, 0.0117, 0.0144,
        0.3360, 0.0342])
tensor([0.0045, 0.0069, 0.0380, 0.0068, 0.0475, 0.1233, 0.0354, 0.0832, 0.0463,
        0.0229, 0.0486, 0.0456, 0.0153, 0.0123, 0.0321, 0.0352, 0.0117, 0.0144,
        0.3360, 0.0342])


OK, then now let's look at the numerical stability of softmax. First we can observe a interesting property of the softmax function: its output is shift/translation invariant:

In [109]:
x = torch.randn(8)
print(naive_softmax(x))
print(naive_softmax(x+5))
print(naive_softmax(x-3))

tensor([0.0351, 0.1232, 0.2528, 0.0892, 0.0902, 0.0238, 0.3785, 0.0074])
tensor([0.0351, 0.1232, 0.2528, 0.0892, 0.0902, 0.0238, 0.3785, 0.0074])
tensor([0.0351, 0.1232, 0.2528, 0.0892, 0.0902, 0.0238, 0.3785, 0.0074])


This porperty allows us to deal with problematic large inputs simply by subtracting their maximum:

In [110]:
def stable_softmax(x):
    m = x.max()
    return (x-m).exp() / (x-m).exp().sum()

This "stable" function now can also deal with larger value that were problematic for our naive function:

In [111]:
large_input = torch.randn(10) * 100

print("naive: ", naive_softmax(large_input))
print("stable: ", stable_softmax(large_input))
print("torch: ", torch.softmax(large_input, dim=-1))


naive:  tensor([3.3718e-20, 1.9575e-20, 4.5934e-35, 1.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00])
stable:  tensor([3.3718e-20, 1.9575e-20, 4.5934e-35, 1.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00])
torch:  tensor([3.3718e-20, 1.9575e-20, 4.5934e-35, 1.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00])


In [112]:
def stable_softmax2(x):
    """returns softmax result and log sum exp"""
    m = x.max()
    a = (x - m).exp()
    b = a.sum()
    lse = m + torch.log(b)
    return a / b, lse

Again we can now use this to combine two softmax block results, but to do it in the same way as before we would need to calculate the exp() values.. which is as we know numerically not stable:

In [113]:
x = torch.randn(20)

a = torch.softmax(x, dim=-1)

x1, x2 = x.chunk(2)

b1, lse1 = stable_softmax2(x1)
b2, lse2 = stable_softmax2(x2)

c1 = b1 * torch.exp(lse1) / (torch.exp(lse1) + torch.exp(lse2))
c2 = b2 * torch.exp(lse2) / (torch.exp(lse1) + torch.exp(lse2))

print(a)
print(torch.cat([c1, c2]), torch.allclose(a, torch.cat([c1, c2])))


tensor([0.0242, 0.0231, 0.0421, 0.0112, 0.0095, 0.5461, 0.0210, 0.0580, 0.0072,
        0.0119, 0.0078, 0.0183, 0.0221, 0.0801, 0.0042, 0.0070, 0.0102, 0.0155,
        0.0381, 0.0422])
tensor([0.0242, 0.0231, 0.0421, 0.0112, 0.0095, 0.5461, 0.0210, 0.0580, 0.0072,
        0.0119, 0.0078, 0.0183, 0.0221, 0.0801, 0.0042, 0.0070, 0.0102, 0.0155,
        0.0381, 0.0422]) True


But luckily log & exp are to the rescue:

In [114]:
d1 = b1 * torch.exp(-torch.log(1 + torch.exp(lse2 - lse1)))
d2 = b2 * torch.exp(-torch.log(1 + torch.exp(lse1 - lse2)))
print(a)
print(torch.cat([d1, d2]), torch.allclose(a, torch.cat([d1, d2])))

tensor([0.0242, 0.0231, 0.0421, 0.0112, 0.0095, 0.5461, 0.0210, 0.0580, 0.0072,
        0.0119, 0.0078, 0.0183, 0.0221, 0.0801, 0.0042, 0.0070, 0.0102, 0.0155,
        0.0381, 0.0422])
tensor([0.0242, 0.0231, 0.0421, 0.0112, 0.0095, 0.5461, 0.0210, 0.0580, 0.0072,
        0.0119, 0.0078, 0.0183, 0.0221, 0.0801, 0.0042, 0.0070, 0.0102, 0.0155,
        0.0381, 0.0422]) True


To understand why `b1 * torch.exp(lse1) / (torch.exp(lse1) + torch.exp(lse2))` is equal to `b1 * torch.exp(-torch.log(1 + torch.exp(lse2-lse1)))` we remember school math basics:

In [115]:
a = 5
b = 3

print("math.exp(5)/math.exp(3) =", math.exp(5) / math.exp(3))
print("math.exp(5 - 3) =", math.exp(5 - 3))

print("a/(a+b) =", a / (a+b))
print("1/(1+b/a) =", 1 / (1+b/a))

math.exp(5)/math.exp(3) = 7.38905609893065
math.exp(5 - 3) = 7.38905609893065
a/(a+b) = 0.625
1/(1+b/a) = 0.625


With the fresh knowledge about softmax we can now take a look at the `update()` function that is used in the ring-flash-attention implementation:

In [116]:
def _update_out_and_lse(
    out: torch.Tensor,
    lse: torch.Tensor,
    block_out: torch.Tensor,
    block_lse: torch.Tensor,
) -> Tuple[torch.Tensor, torch.Tensor]:
    block_out = block_out.to(torch.float32)
    block_lse = block_lse.transpose(-2, -1).unsqueeze(dim=-1)

    new_lse = lse + torch.log(1 + torch.exp(block_lse - lse))
    out = torch.exp(lse - new_lse) * out + torch.exp(block_lse - new_lse) * block_out

    lse = new_lse
    return out, lse